# Bikeability of Seoul

In [ ]:
import osmnx as ox
import networkx as nx
import pandana
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cm as cm
import seaborn as sns
import folium 

%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

In [ ]:
city = ox.geocode_to_gdf("Seoul, KR")

In [ ]:
city_proj = ox.project_gdf(city)
print(city_proj.crs)

In [ ]:
ax = city_proj.plot(fc="grey")
_ = ax.axis("off")

In [ ]:
place = 'Seoul, KR'

graph = ox.graph_from_place(place, network_type="bike") 

In [ ]:
Seoul_street = ox.plot_graph_folium(graph, popup_attribute="name", weight=0.5, color="#8b0000", opacity=0.7)
Seoul_street

In [ ]:
nodes, streets = ox.graph_to_gdfs(graph)
streets.head()

In [ ]:
type(streets)

In [ ]:
streets.columns

In [ ]:
cityname = 'Seoul, KR'

In [ ]:
graph_bike = ox.graph_from_place(cityname, network_type="bike")

In [ ]:
Seoul_street = ox.plot_graph_folium(graph, popup_attribute="name", weight=0.5, color="#8b0000", opacity=0.7)
Seoul_street

In [ ]:
graph_bike = ox.projection.project_graph(graph_bike, to_crs=5179)

In [ ]:
tags_bike = {
    'amenities':[
        'bicycle_rental=docking_station',
        'bicycle_rental=dropoff_point'
    ]
}

In [ ]:
pois_bike = ox.geometries.geometries_from_place(cityname, tags=tags_bike)

In [ ]:
pois_bike = pois.to_crs(epsg=5179)
pois_bike

In [ ]:
# Max time to walk in minutes (no routing to nodes further than this)
bike_time = 15

# Walking speed
bike_speed = 20

In [ ]:
# Set a uniform walking speed on every edge
for u, v, data in graph_bike.edges(data=True):
    data['speed_kph'] = bike_speed
graph_bike = ox.add_edge_travel_times(graph_bike)

# Extract node/edge GeoDataFrames, retaining only necessary columns (for pandana)
nodes_bike = ox.graph_to_gdfs(graph_bike, edges=False)[['x', 'y']]
edges_bike = ox.graph_to_gdfs(graph_bike, nodes=False).reset_index()[['u', 'v', 'travel_time']]

In [ ]:
centroids_bike = pois_bike.centroid

In [ ]:
network = pandana.network.Network(
    node_x=nodes_bike['x'],
    node_y=nodes_bike['y'], 
    edge_from=edges_bike['u'],
    edge_to=edges_bike['v'],
    edge_weights=edges_bike[['travel_time']]
)

In [ ]:
centroids_bike = pois_bike.centroid

In [ ]:
# Minutes -> seconds
maxdist = bike_time * 60

In [ ]:
network.set_pois(
    category='pois',
    maxdist=maxdist,
    maxitems=5,
    x_col=centroids_bike.x, 
    y_col=centroids_bike.y
)

In [ ]:
distances = network.nearest_pois(
    distance=maxdist,
    category='pois',
    num_pois=5
)

distances.astype(int).head()

In [ ]:
# Set text parameters
COLOR = 'white'
plt.rcParams['text.color'] = COLOR
plt.rcParams['axes.labelcolor'] = COLOR
plt.rcParams['xtick.color'] = COLOR
plt.rcParams['ytick.color'] = COLOR

# Setup plot
fig, ax = plt.subplots(figsize=(20,15))
ax.set_axis_off()
ax.set_aspect('equal')
fig.set_facecolor((0,0,0))

# Plot distance to nearest POI
sc = ax.scatter(
    x=nodes_bike['x'],
    y=nodes_bike['y'], 
    c=distances[1],
    s=5,
    cmap='viridis_r',
)

# Colorbar
cb = fig.colorbar(sc, ax=ax, shrink=0.8, ticks=[0, 300, 600, 900])
cb.ax.tick_params(color='none', labelsize=20)
cb.ax.set_yticklabels(['0', '5', '10', '>= 15'])
cb.set_label('Biking time to nearest POI (minutes)', fontsize=20, fontweight='bold')

# Remove empty space
plt.tight_layout()

# Ttareungi Public Bicycle Heat Map 

In [ ]:
import json
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import folium
from folium import plugins
import missingno as mnso

import warnings
warnings.filterwarnings('ignore')

geo_path = 'data/seoul_municipalities_geo_simple.json'
geo_str = json.load(open(geo_path, encoding='utf-8'))

plt.rcParams['axes.unicode_minus'] = False
plt.rcParams["font.family"] = 'NanumGothic'

In [ ]:
rental = pd.read_csv('data/서울특별시 공공자전거 대여소 정보.csv')
rental.head()

In [ ]:
rental.shape

In [ ]:
import missingno as msno

msno.matrix(rental)

In [ ]:
bike_map = folium.Map(location=[rental['위도'].mean(), rental['경도'].mean()], zoom_start=10.8, tiles='CartoDB positron')
for i, row in rental.iterrows():
    folium.CircleMarker(
        location=[row['위도'], row['경도']],
        radius=1,
        fill=True,
    ).add_to(bike_map)
bike_map


In [ ]:
bike_map = folium.Map(location=[rental['위도'].mean(), rental['경도'].mean()], zoom_start=10.8, tiles='CartoDB positron')
plugins.HeatMap(rental[['위도', '경도']].values.tolist(),
                        radius=10, blur=5
                        ).add_to(bike_map)
plugins.Fullscreen(positions='topright').add_to(bike_map)
bike_map
